# LangChain 0.1.0 Quickstart - Create Your Own RAG Chains

Welcome to this Quickstart walkthrough of the new version of Langchain. This tutorial follows the official Langchain documentation quickstart. I made a video where I go through exactly all the code in this notebook 👉 https://youtu.be/LBNpyjcbv0o

# 1.0 Install the required packages

Some packages have changed inside Langchain. This is how you should install and import them from now on. Remember that langchain has been divided into three main packages:
- `langchain` contains pre-built chains and other stuff you may need.
- `langchain-core` contains the main abstractions of the library.
- `langchain-community` contains third-party integrations like Chroma, FAISS, etc.
- Some third-party integrations have their own package (outside `langchain-community`) that you should install if you want to use them. For example, OpenAI has its own package: `langchain-openai`.

All three are installed when you run `pip install langchain`. But you can install community or core individually as well.

In [ ]:
! pip install langchain
! pip install langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.6/229.6 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
llm.invoke("what is docker and how is it useful for deployment?")

AIMessage(content="Docker is an open-source platform that allows developers to automate the deployment and management of applications within lightweight, isolated containers. It provides an efficient way to package an application and all its dependencies into a single standardized unit, called a container. These containers are portable and can be run consistently across different environments, such as development, testing, and production.\n\nHere are some key benefits of using Docker for deployment:\n\n1. Portability: Docker containers encapsulate the application and its dependencies, ensuring consistency between different environments. This portability allows for easy deployment across different servers, cloud platforms, or even on local machines.\n\n2. Isolation: Each Docker container runs in its own isolated environment, separate from other containers and the host machine. This ensures that applications are isolated from one another, preventing conflicts and providing a secure runti

# 2.0 Create your first chain with LCEL

LCEL is now the default way to create chains in LangChain. It has a more pipeline-like syntax and allows you to modify already-existing chains.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an English-French translator that return whatever the user says in French"),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({
    "input": "i enjoy going to rock concerts"
    })

AIMessage(content="j'aime aller aux concerts de rock")

In [ ]:
# add output parser to the chain

from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "my friend robert has a blue cat"})

'mon ami robert a un chat bleu'

In [ ]:
llm.invoke("what is new in langchain 0.1.0?")

AIMessage(content='I\'m sorry, but I couldn\'t find any specific information about a software or product called "Langchain 0.1.0." It is possible that it is a lesser-known or niche product, or it might not exist. Could you please provide more context or clarify your question?')

# 3.0 Create a Retrieval Chain


## 3.1 Load the source documents

First, we will have to load the documents that will enrich our LLM prompt. We will use [this blog post](https://blog.langchain.dev/langchain-v0-1-0/) from LangChain's official website explaining the new release. OpenAI's models were not trained on this content, so the only way to ask questions about it is to build a RAG chain.

The first thing to do is to load the blog content to our vector store. We will use beautiful soup to scrap the blog post. Then we will store it in a FAISS vector store.

In [ ]:
# retrieval chain

! pip install beautifulsoup4

In [ ]:
! pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langchain-v0-1-0/")

docs = loader.load()

In [ ]:
docs

[Document(page_content='\n\n\nLangChain v0.1.0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain v0.1.0\n\n10 min read\nJan 8, 2024\n\n\n\n\n\nToday we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äô

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter  import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)


In [ ]:
documents

[Document(page_content='LangChain v0.1.0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangChain v0.1.0\n\n10 min read\nJan 8, 2024', metadata={'source': 'https://blog.langchain.dev/langchain-v0-1-0/', 'title': 'LangChain v0.1.0', 'language': 'en'}),
 Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS

In [ ]:
vectorstore = FAISS.from_documents(documents, embeddings)

## 3.2 Create a Context-Aware LLM Chain

Here we create a chain that will answer a question given a context. For now, we are passing the context manually, but in the next step we will pass in the documents fetched from the vector store we created above 👆

In [ ]:
# create chain for documents

from langchain.chains.combine_documents import create_stuff_documents_chain

template = """"Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""
prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "what is langchain 0.1.0?",
    "context": [Document(page_content="langchain 0.1.0 is the new version of a llm app development framework.")]
})

'Langchain 0.1.0 is the new version of a llm app development framework.'

## 3.2 Create the RAG Chain

RAG stands for Retrieval-Augmented Generation. This means that we will enrich the prompt that we send to the LLM. We will use with the documents that wil will retrieve from the vector store for this. LangChain comes with the function `create_retrieval_chain` that allows you to create one of these.

In [ ]:
# create retrieval chain

from langchain.chains import create_retrieval_chain

retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({
    "input": "what is new in langchain 0.1.0"
})

In [ ]:
response['answer']

'In langchain 0.1.0, the new features include improved focus through both functionality and documentation, a new versioning standard, the separation of langchain-core and partner packages, and the introduction of LangSmith for debugging LLM applications.'

# 4.0 Create Conversational RAG Chain

Now we will create exactly the same thing as above, but we will have the AI assistant take the history of the conversation into account. In short, we will build the same chain as above but with we will take into account the previous messages in these two steps of the chain:

- When fetching the documents from the vector store. We will fetch documents related to the entire conversation and not just the latest message.
- When answering the question. We will send to the LLM the history of the conversation along the context and query.





## 4.1 Create a Conversation-Aware Retrieval Chain

This chain will return the documents related to the entire conversation and not just the latest message.

In [ ]:
# conversational retrieval chain

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me more about it!"
})

[Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Specifically we made two large architectural changes: separating out langchain-core and separating out partner packages (either into langchain-community or standalone p

## 4.2 Use Retrieval Chain together with Document Chain

Now we will create a document chain that contains a placeholder for the conversation history. This placeholder will be populated with the conversation history that we will pass as its value. We will the plug it together with the retriever chain we created above to have a conversational retrieval-augmented chain.

In [ ]:
from langchain.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

conversational_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
response = conversational_retrieval_chain.invoke({
    'chat_history': [],
    "input": "What is langchain 0.1.0 about?"
})

In [ ]:
response

{'chat_history': [],
 'input': 'What is langchain 0.1.0 about?',
 'context': [Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Specifically we made two large architectural changes: separating out langchain-core and sep

In [ ]:
response['answer']

'LangChain 0.1.0 is the first stable version of the LangChain framework. It is fully backwards compatible and available in both Python and JavaScript. This release focuses on improving functionality and documentation. The stable version of LangChain aims to earn developer trust and provide a foundation for systematic and safe evolution of the library.'

In [ ]:
# simulate conversation history

chat_history = [
    HumanMessage(content="Is there anything new about Langchain 0.1.0?"),
    AIMessage(content="Yes!")
]

response = conversational_retrieval_chain.invoke({
    'chat_history': chat_history,
    "input": "Tell me more about it!"
})

In [ ]:
response

{'chat_history': [HumanMessage(content='Is there anything new about Langchain 0.1.0?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me more about it!',
 'context': [Document(page_content='Today we‚Äôre excited to announce the release of langchain 0.1.0, our first stable version. It is fully backwards compatible, comes in both Python and JavaScript, and comes with improved focus through both functionality and documentation. A stable version of LangChain helps us earn developer trust and gives us the ability to evolve the library systematically and safely.Python GitHub DiscussionPython v0.1.0 GuidesJS v0.1.0 GuidesYouTube WalkthroughIntroductionLangChain has been around for a little over a year and has changed a lot as it‚Äôs grown to become the default framework for building LLM applications. As we previewed a month ago, we recently decided to make significant changes to the\xa0 LangChain package architecture in order to better organize the project and strengthen the foundation.\xa0Sp

In [ ]:
response['answer']

"LangChain 0.1.0 is the first stable version of the LangChain framework. It is fully backwards compatible and is available in both Python and JavaScript. This release comes with improved functionality and documentation, providing a more focused and reliable experience for developers. \n\nOne significant change in LangChain 0.1.0 is the separation of langchain-core and partner packages. Langchain-core contains the main abstractions, interfaces, and core functionality, while partner packages are either included in langchain-community or exist as standalone partner packages. This architectural change helps organize the project and strengthen its foundation. \n\nTo address the challenges posed by previous minor version releases (0.0.x), LangChain now follows a new versioning standard. Any breaking changes to the public API will result in a minor version bump (second digit), while bug fixes and new features will result in a patch version bump (third digit). This versioning standard aims to 

# Conclusion

Congratulations for finishing this tutorial. For a more in-depth walk-through, you can go to the official [LangChain documentation](https://python.langchain.com/docs/get_started/quickstart).

In the meantime, be sure to subscribe to [my YouTube channel](https://www.youtube.com/@alejandro_ao). I upload free tutorials and courses to help you become an AI Engineer.